<a href="https://colab.research.google.com/github/Haresh-16/LLM-Course-EE-596---Mini-Project-1/blob/master/miniproject1_part2_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Team Name - Leo x Vikram <br>
Done by Haresh Indrajit and Justin-Pierre Trembley

In [1]:
from huggingface_hub import login
login()

In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=eb1452dc607d99b3a2ce79a429f6fe25c6290a7894fd051c790ae6cb5dfcc2b8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00


In [12]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset

class TextSimilarityModel:
    def __init__(self, corpus_name, rel_name, model_name='all-MiniLM-L6-v2', top_k=10):
        """
        Initialize the model with datasets and pre-trained sentence transformer.
        """
        self.model = SentenceTransformer(model_name)
        self.corpus_name = corpus_name
        self.rel_name = rel_name
        self.top_k = top_k
        self.load_data()

    def load_data(self):
        """
        Load and filter datasets based on test queries and documents.
        """
        # Load query and document datasets
        dataset_queries = load_dataset(self.corpus_name, "queries")
        dataset_docs = load_dataset(self.corpus_name, "corpus")

        # Extract queries and documents
        self.queries = dataset_queries["queries"]["text"]
        self.query_ids = dataset_queries["queries"]["_id"]
        self.documents = dataset_docs["corpus"]["text"]
        self.document_ids = dataset_docs["corpus"]["_id"]

        # Buil query and document id to text mapping
        self.query_id_to_text = {query_id:query for query_id, query in zip(self.query_ids, self.queries)}
        self.document_id_to_text = {document_id:document for document_id, document in zip(self.document_ids, self.documents)}

        # Build relevant queries and documents mapping based on train set
        train_qrels = load_dataset(self.rel_name)["train"]
        self.train_query_id_to_relevant_doc_ids = {qid: [] for qid in train_qrels["query-id"]}
        for qid, doc_id in zip(train_qrels["query-id"], train_qrels["corpus-id"]):
            if qid in self.train_query_id_to_relevant_doc_ids:
                self.train_query_id_to_relevant_doc_ids[qid].append(doc_id)

        # Filter queries and documents and build relevant queries and documents mapping based on test set
        test_qrels = load_dataset(self.rel_name)["test"]
        self.filtered_test_query_ids = set(test_qrels["query-id"])
        self.filtered_test_doc_ids = set(test_qrels["corpus-id"])

        self.test_queries = [q for qid, q in zip(self.query_ids, self.queries) if qid in self.filtered_test_query_ids]
        self.test_query_ids = [qid for qid in self.query_ids if qid in self.filtered_test_query_ids]
        self.test_documents = [doc for did, doc in zip(self.document_ids, self.documents) if did in self.filtered_test_doc_ids]
        self.test_document_ids = [did for did in self.document_ids if did in self.filtered_test_doc_ids]


        self.test_query_id_to_relevant_doc_ids = {qid: [] for qid in self.test_query_ids}
        for qid, doc_id in zip(test_qrels["query-id"], test_qrels["corpus-id"]):
            if qid in self.test_query_id_to_relevant_doc_ids:
                self.test_query_id_to_relevant_doc_ids[qid].append(doc_id)

    def rank_documents(self):
        """
        Ranks documents for each query using the pre-trained model.
        (1) Compute cosine similarity between each document and the query
        (2) Rank documents for each query and save the results in a dictionary "query_id_to_ranked_doc_ids"
            This will be used in "mean_average_precision"
            Example format {2: [125, 673], 35: [900, 822]}
        """
        query_embeddings = self.model.encode(self.test_queries)
        document_embeddings = self.model.encode(self.test_documents)

        self.query_id_to_ranked_doc_ids = {}
        #TODO Put your code here.
        ###########################################################################
        similarities = []

        # Compute cosine similarity between each document and the query
        for i, query_embedding in enumerate(query_embeddings):
            try:
                similarities = cosine_similarity([query_embedding], document_embeddings)[0]
                # Get the indices of the documents in descending order of similarity

                #print("similarities: ",similarities)
                ranked_doc_ids = np.argsort(similarities)[::-1][:self.top_k]

                # Save the ranked document IDs for this query
                #query_id_to_ranked_doc_ids = [lst[i] for i in indices]
                self.query_id_to_ranked_doc_ids[self.test_query_ids[i]] = [self.test_document_ids[j] for j in ranked_doc_ids]
            except:
                continue
        ###########################################################################

    @staticmethod
    def average_precision(relevant_docs, candidate_docs):
        """
        Compute average precision for a single query.
        """
        y_true = [1 if doc_id in relevant_docs else 0 for doc_id in candidate_docs]
        precisions = [np.mean(y_true[:k+1]) for k in range(len(y_true)) if y_true[k]]
        return np.mean(precisions) if precisions else 0

    def mean_average_precision(self):
        """
        Compute mean average precision for all queries using the "average_precision" function.
        A reference: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52
        """
        #TODO Put your code here.
        ###########################################################################
        average_precisions = []

        # For each query, compute the average precision
        for i,query_id in enumerate(self.test_query_ids):
            ap = TextSimilarityModel.average_precision(self.test_query_id_to_relevant_doc_ids[query_id], self.query_id_to_ranked_doc_ids[query_id])
            average_precisions.append(ap)

        # Compute the mean average precision
        map_score = np.mean(average_precisions) if average_precisions else 0

        return map_score
        ###########################################################################


    def fine_tune_model(self, batch_size=32, num_epochs=3, save_model_path="finetuned_senBERT"):
        """
        Fine-tunes the model using MultipleNegativesRankingLoss.
        (1) Load training data
        (2) Prepare training examples
        (3) Define a loss function
        (4) Fine-tune the model
        """
        #TODO Put your code here.
        ###########################################################################
        #train_mapping_ids = self.load_dataset(self.rel_name)["train"]

        # Prepare training examples
        train_examples = self.prepare_training_examples()

        # Define a loss function
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
        train_loss = losses.MultipleNegativesRankingLoss(self.model)

        # Fine-tune the model
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=num_epochs,
            warmup_steps=100,
            output_path=save_model_path
        )
        ###########################################################################


    def prepare_training_examples(self):#, train_mapping_ids):
        """
        Prepares training examples from the training data.
        """
        train_examples = []
        used_doc_ids = []
        for qid, doc_ids in self.train_query_id_to_relevant_doc_ids.items():
            usable_doc_ids = set(doc_ids) - set(used_doc_ids)
            for doc_id in usable_doc_ids:
                anchor = self.query_id_to_text[qid]
                positive = self.document_id_to_text[doc_id]
                train_examples.append(InputExample(texts=[anchor, positive]))
            used_doc_ids.extend(doc_ids)

        '''
        MAYBE USEFUL FOR FINE-TUNING

        test_examples = []
        for qid, doc_ids in self.test_query_id_to_relevant_doc_ids.items():
            usable_doc_ids = set(doc_ids) - set(used_doc_ids)
            for doc_id in usable_doc_ids:
                anchor = self.query_id_to_text[qid]
                positive = self.document_id_to_text[doc_id]
                test_examples.append(InputExample(texts=[anchor, positive]))
        '''
        # test_query_id_to_relevant_doc_ids - For seeing Test Loss during Training!

        return train_examples


# Initialize the model

#model = TextSimilarityModel("BeIR/nfcorpus", "BeIR/nfcorpus-qrels", model_name='finetuned_senBERT_train')
model_names=['all-MiniLM-L6-v2','all-mpnet-base-v2']
model = TextSimilarityModel("BeIR/nfcorpus", "BeIR/nfcorpus-qrels", model_name='all-MiniLM-L6-v2')#model_name=model_names[1]) #top_k=20

# Finetune the model
model.fine_tune_model(batch_size=64, num_epochs=10, save_model_path="finetuned_senBERT_train_v2")  # Adjust batch size and epochs as needed

model.rank_documents()
map_score = model.mean_average_precision()
print("Mean Average Precision:", map_score)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57 [00:00<?, ?it/s]

Mean Average Precision: 0.4485820566043181
